In [11]:
import numpy as np 
import matplotlib.pyplot as plt

# add the gridding path 
import sys
sys.path.append("/home/ian/Research/Disks/MillionPoints/million-points-of-light")
sys.path.append("/Users/ianczekala/Documents/ALMA/million-points-of-light")

import gridding

# convert from arcseconds to radians
arcsec = np.pi / (180.0 * 3600) # [radians]  = 1/206265 radian/arcsec

In [12]:
def sky_plane(alpha, dec, a=1, delta_alpha=1.0*arcsec, delta_delta=1.0*arcsec, sigma_alpha=1.0*arcsec,
              sigma_delta=1.0*arcsec, Omega=0.0):
    '''
    alpha: ra (in radians)
    delta: dec (in radians)
    a : amplitude
    delta_alpha : offset (in radians)
    delta_dec : offset (in radians)
    sigma_alpha : width (in radians)
    sigma_dec : width (in radians)
    Omega : position angle of ascending node (in degrees east of north)
    '''

    return a * np.exp(-( (alpha - delta_alpha)**2/(2 * sigma_alpha**2) + \
                        (dec - delta_delta)**2/(2 * sigma_delta**2)))


def fourier_plane(u, v, a=1, delta_alpha=1.0*arcsec, delta_delta=1.0*arcsec, sigma_alpha=1.0*arcsec,
              sigma_delta=1.0*arcsec, Omega=0.0):
    '''
    Calculate the Fourier transform of the Gaussian. Assumes u, v in kλ.
    '''

    # convert back to lambda
    u = u * 1e3
    v = v * 1e3

    return 2 * np.pi * a * sigma_alpha * sigma_delta * np.exp(- 2 * np.pi**2 * \
                (sigma_alpha**2 * u**2 + sigma_delta**2 * v**2) - 2 * np.pi * 1.0j * \
                                                    (delta_alpha * u + delta_delta * v))

def fftspace(width, N):
    '''Oftentimes it is necessary to get a symmetric coordinate array that spans ``N``
     elements from `-width` to `+width`, but makes sure that the middle point lands
     on ``0``. The indices go from ``0`` to ``N -1.``
     `linspace` returns  the end points inclusive, wheras we want to leave out the
     right endpoint, because we are sampling the function in a cyclic manner.'''

    assert N % 2 == 0, "N must be even."

    dx = width * 2.0 / N
    xx = np.empty(N, np.float)
    for i in range(N):
        xx[i] = -width + i * dx
    
    return xx

In [13]:
# Let's plot this up and see what it looks like 

N_alpha = 128
N_dec = 128
img_radius = 15.0 * arcsec


# full span of the image
ra = fftspace(img_radius, N_alpha) # [arcsec]
dec = fftspace(img_radius, N_dec) # [arcsec]

# calculate the maximum u and v points that our image grid can sample 
dRA = (2 * img_radius) / N_alpha # radians
max_baseline = 1 / (2 * dRA) * 1e-3 # kilolambda, nyquist rate
print(max_baseline) # kilolambda

440.03158666047227


In [18]:
# create some fake data

np.random.seed(42)

N_vis = 100 # number of data points 

# the fake baselines where the Visibility function is sampled 
u_data = np.random.normal(loc=0, scale=0.1 * max_baseline, size=N_vis)
v_data = np.random.normal(loc=0, scale=0.1 * max_baseline, size=N_vis)

data_points = np.array([u_data, v_data]).T

# create a dataset of a Gaussian
data_only = fourier_plane(u_data, v_data)

# add some noise
noise = 1e-12 * np.ones(N_vis) # Jy
noise_draw = np.random.normal(loc=0, scale=noise, size=(N_vis)) + \
    np.random.normal(loc=0, scale=noise, size=(N_vis)) * 1.0j 
data_values = data_only + noise_draw

In [19]:
# create fixed quantities that we can pre-calculate in numpy before stuffing into the Theano part 

# the image plane grid (fixed throughout the problem)
XX, YY = np.meshgrid(np.fft.fftshift(ra), np.fft.fftshift(dec))

# the image-plane taper (fixed throughout problem)
corrfun_mat = gridding.corrfun_mat(np.fft.fftshift(ra), np.fft.fftshift(dec))

# the u and v coordinates of the RFFT output (also fixed throughout problem)
u_coords = np.fft.rfftfreq(N_alpha, d=(2 * img_radius)/N_alpha) * 1e-3  # convert to [kλ]
v_coords = np.fft.fftfreq(N_dec, d=(2 * img_radius)/N_dec) * 1e-3  # convert to [kλ]

# calculate the C_real and C_imag matrices
# these are scipy csc sparse matrices that will be stuffed into Theano objects
C_real, C_imag = gridding.calc_matrices(data_points, u_coords, v_coords)

In [21]:
np.save("data_points.npy", data_points)
np.save("noise.npy", noise)
np.save("data_real.npy", np.real(data_values))
np.save("data_imag.npy", np.imag(data_values))

np.save("corrfun.npy", corrfun_mat)

np.save("C_real.npy", C_real.toarray())
np.save("C_imag.npy", C_imag.toarray())